# Assignment 3

Due: Sunday, October 6, 2019

Please complete all parts of this assignment within this notebook. You may either leave the notebook file on the JupyterHub server (where it will be collected automatically on the due date) or it may be uploaded to the D2L Dropbox if you choose not to work on the JupyterHub server.

Use both Python code cells and Jupyter Markdown cells to complete the following exercises. Place your solution immediately after the corresponding question.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive

- - -

### A. **Problem 3.10** Trajectory of a steel ball

*The first two parts of this problem (a) and (b) were completed as part of Lecture 5 and the code re

In [3]:
### Code from Lecture 5 reused in this problem:

g = 9.81

def ProjectileMotionStepper(dt=0.01,
                            m=1, C2=0.005,
                            x0=0, y0=0, 
                            vx0=30, vy0=30,
                            tmax=6):
    
    # initialize the model
    t = 0
    x = x0
    y = y0
    vx = vx0
    vy = vy0
    
    while True:
        
        model = {'t': t, 
                 'x': x, 'y': y,
                 'vx': vx, 'vy': vy}
        yield model # return the model state back to the caller
    
        if t > tmax:
            break
        if y < 0: # stop if the projectile hits the ground
            break
            
        # drag forces
        v = np.sqrt(vx**2+vy**2)
        Fdx = -C2*v*vx
        Fdy = -C2*v*vy
        
        # use the Euler algorithm to update the state of the model
        x = x + vx*dt
        y = y + vy*dt
        vx = vx + Fdx/m * dt
        vy = vy + (Fdy/m - g) * dt
        t = t + dt
        
def ProjectileMotionPlot(particle, style='', legend=True):
    # style is optional parameter to control color, linestyle, and marker used
    
    plt.plot(particle.x, particle.y, style, label='Numerical')
    plt.ylabel('y (m)')
    plt.xlabel('x (m)')
    
    # extract out initial conditions
    init = particle.iloc[0]
    x0 = init.x
    y0 = init.y
    vx0 = init.vx
    vy0 = init.vy
    g = 9.8
   
    # Analytical solution
    x = particle.x
    y = y0 + vy0/vx0 * (x-x0) - 1/2 * g/vx0**2 * (x-x0)**2
    plt.plot(x, y, style, label='Analytical')
    
    # draw the ground level, y=0
    plt.axhline(0, color='k')
    
    # ensure aspect ratio is square
    plt.axis('equal')
    if legend:
        plt.legend()
        


a. Compute the two-dimensional trajectory of a ball moving in air without air friction, and plot $y$ as a function of $x$. 

Compare your computed results with the exact results. For example, assume that a ball is thrown from ground level at an angle $\theta_0$ above the horizontal with an initial velocity of $v_0= 15\;\mbox{m/s}$. 

Vary $\theta_0$ and show that the maximum range occurs at $\theta_0=\theta_{max}=45^\circ$. What is $R_{max}$, the maximum range, at this angle? Compare your numerical result to the analytical result $R_{max}=v_0^2/g$.

In [ ]:

def Problem310aApp(angle=45):
   
    v0 = 15
    θ0 = np.deg2rad(angle)
    
    vx0 = v0 * np.cos(θ0)
    vy0 = v0 * np.sin(θ0)
   
    # set up the model
    model = ProjectileMotionStepper(dt=0.01, 
                                    y0=0, 
                                    vx0=vx0, vy0=vy0, 
                                    C2=0.0)
    
    # iterate the model
    ball = pd.DataFrame(model)
    
    # plot the results
    ProjectileMotionPlot(ball, style='-')

    # determine maximum range
    # use max to determine max range?
    #Rmax = max(ball.x) 
    # average of last to points?
    #Rmax = (ball.x.iloc[-1] + ball.x.iloc[-2])/2
    # linear interpolation: y-y1 = (y2-y1)/(x2-x1)*(x-x1)
    x1 = ball.x.iloc[-2]
    x2 = ball.x.iloc[-1]
    y1 = ball.y.iloc[-2]
    y2 = ball.y.iloc[-1]
    # solve for x: x-x1 = (x2-x1)/(y2-y1)*(y-y1)
    # evaluate at = 0:
    R = x1 + (x2-x1)/(y2-y1)*(0-y1)
    Rmax = v0**2/g
    
    plt.axvline(R, color='C0')
    plt.axvline(Rmax, color='C1')
    print('R = {:.2f} Rmax = {:.2f}'.format(R, Rmax))
       
interactive(Problem310aApp, angle=(0,90,5))

b. Suppose that a steel ball is thrown from a height $h$ at an angle $\theta_0$ above the horizontal with the same initial speed as in part (a). If you neglect air resistance, do you expect $\theta_{max}$ to be larger or smaller than $45^\circ$? What is $\theta_{max}$ for $h= 2\;\mbox{m}$? By what percent is the range $R$ changed if $\theta$ is varied by 2% from $\theta_{max}$?

In [ ]:
def Problem310bApp(h=2):
   
    v0 = 15
    angles = np.arange(0, 90, 5)
    ranges = []
    
    for angle in angles:
        θ0 = np.deg2rad(angle)

        vx0 = v0 * np.cos(θ0)
        vy0 = v0 * np.sin(θ0)

        # set up the model
        model = ProjectileMotionStepper(dt=0.01, 
                                        y0=h, 
                                        vx0=vx0, vy0=vy0, 
                                        C2=0.0)

        # iterate the model
        ball = pd.DataFrame(model)

        # determine maximum range
        x1 = ball.x.iloc[-2]
        x2 = ball.x.iloc[-1]
        y1 = ball.y.iloc[-2]
        y2 = ball.y.iloc[-1]
        # solve for x: x-x1 = (x2-x1)/(y2-y1)*(y-y1)
        # evaluate at = 0:
        R = x1 + (x2-x1)/(y2-y1)*(0-y1)
        
        ranges.append(R)
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].plot(angles, ranges, 'o-')
    axs[0].set_xlabel('Angle')
    axs[0].set_ylabel('Range (m)')
    
    # need to figure out angle that gives the max range
    # maximum occurs when derivative is zero, dR/da = 0
    dR = np.diff(ranges)
    da = np.diff(angles)
    dRda = dR / da
    a = (angles[1:]+angles[:-1])/2
    
    axs[1].plot(a, dRda, 'o-')
    axs[1].set_xlabel('Angle')
    axs[1].set_ylabel('dR/da')
    
    # root finding problem
    for i in range(len(ranges)-1):
        y1 = dRda[i]
        y2 = dRda[i+1]
        x1 = a[i]
        x2 = a[i+1]
        if y1 > 0 and y2 <= 0:
            break
    # linear interpolation
    angle_max = x1 + (x2-x1)/(y2-y1)*(0-y1)    
    
    axs[1].axhline(0)
    axs[1].axvline(angle_max)
    print('h = {:.1f} Angle max = {:.2f}'.format(h, angle_max))
        
interactive(Problem310bApp, h=(0, 10, 1))

In [ ]:
def FindProjectileMaxRange(angle=45, h=0):
   
    v0 = 15
    θ0 = np.deg2rad(angle)
    
    vx0 = v0 * np.cos(θ0)
    vy0 = v0 * np.sin(θ0)
   
    # set up the model
    model = ProjectileMotionStepper(dt=0.01, 
                                    y0=0, 
                                    vx0=vx0, vy0=vy0, 
                                    C2=0.0)
    
    # iterate the model
    ball = pd.DataFrame(model)
    
    # determine maximum range
    # use max to determine max range?
    #Rmax = max(ball.x) 
    # average of last to points?
    #Rmax = (ball.x.iloc[-1] + ball.x.iloc[-2])/2
    # linear interpolation: y-y1 = (y2-y1)/(x2-x1)*(x-x1)
    x1 = ball.x.iloc[-2]
    x2 = ball.x.iloc[-1]
    y1 = ball.y.iloc[-2]
    y2 = ball.y.iloc[-1]
    # solve for x: x-x1 = (x2-x1)/(y2-y1)*(y-y1)
    # evaluate at = 0:
    R = x1 + (x2-x1)/(y2-y1)*(0-y1)
    
    print('h = {:.1f} angle = {:.2f} R = {:.2f}'.format(h, angle, R))
    return R

a = 42.64 # max angle for h = 2
R1 = FindProjectileMaxRange(angle=a, h=2)
R2 = FindProjectileMaxRange(angle=a*1.02, h=2)
R3 = FindProjectileMaxRange(angle=a*0.98, h=2)

change_in_R = (R2 - R1)/R1
print('A +2% change in angle caused a {:.2f}% increase in range.'.format(change_in_R*100))
change_in_R = (R3 - R1)/R1
print('A -2% change in angle caused a {:.2f}% decrease in range.'.format(change_in_R*100))

c. Consider the effects of air resistance on the range and optimum angle of a steel ball. For a ball of mass 7 kg and cross-sectional area 0.01 m$^2$, the parameter $C_2 \approx 0.1$. What are the units of $C_2$? It is convenient to exaggerate the effects of air resistance so that you can more easily determine the qualitative nature of the effects. Hence, compute the optimum angle for $h= 2\;\mbox{m}$, $v_0=30\;\mbox{m/s}$, and $C_2/m=0.1$, and compare your answer to the value found in part (b). Is $R$ more or less sensitive to changes in $\theta_0$ from $\theta_{max}$ than in part (b)? Determine the optimum launch angle and the corresponding range for the more realistic value of $C_2= 0.1$. 

- - -
### B. **Problem** Model a ping-pong ball

Model the behaviour of a ping-pong ball when served with either top-spin or back-spin in a game of table tennis.

The drag force can be estimated from 
\begin{align}
F_{drag} \approx - \frac{1}{2} C_d \rho A v^2
\end{align}
with $C_d = \frac{1}{2}$ (since the ball is smooth). 
The Magnus acceleration (force per unit mass) of a ping-pong ball is $C_M \; \vec{\omega} \times \vec{v}$, where $C_M = 0.040$ (in SI units, with $\omega$ in rad/s and $v$ in m/s).  Make an interactive app for different angular velocities in the range -10 to 10  revolutions per second (that is, either top-spin or back-spin) showing the trajectory of a ping pong ball. 
Add to your plot graphical elements showing the net, table, and floor so that your trajectory is seen in the context of the game. Set up your model so that the ping pong ball stops if it hits the net and bounces (elastically, with no energy loss) if it hits the table or the floor. You may also assume no friction between the ball and the table/floor so there are no torques to consider--the angular velocity of the ball remains constant throughout this problem.

You will have to make informed choices and assumptions about the cross-sectional area of a ping-pong ball (remember to not confuse radius with diameter if looking it up on the web), the mass of the ping-pong ball, reasonable velocities when serving a ping-pong ball, height of launching the ball, the size of the table, height of table, and and the height of the net. 

*Giordano 2.24*